In [1]:
!pip install tensorflow
!pip install gpflow==2.9.1
# !pip install gpflow==2.6.3
# !pip install gpflow==2.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
import gpflow
print(gpflow.__version__)

2.9.1


SETTING SMALL VARIANCE

In [ ]:
import gpflow.utilities

In [ ]:
gpflow.utilities.print_summary(model)
gpflow.utilities.print_summary(model.trainable_variables)

╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤═══════════════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │ value             │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪═══════════════════╡
│ GPR.kernel.variance     │ Parameter │ Softplus         │         │ True        │ ()      │ float64 │ 146.23673         │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼───────────────────┤
│ GPR.kernel.lengthscales │ Parameter │ Softplus         │         │ True        │ (2,)    │ float64 │ [1.83272 1.83272] │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼───────────────────┤
│ GPR.likelihood.variance │ Parameter │ Softplus + Shift │         │ True        │ ()      │ float64 │ 146.60784         │
╘═══════════════

In [5]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import gpflow
import pandas as pd

Function to generate Noise:

In [ ]:
def generate_noisy_data(x, n_points=5, noise_std_range=(0.0, 1.0)):
    X_noisy = []
    y_noisy = []
    noise_levels = []

    for point in x:
        noise_level = np.random.uniform(noise_std_range[0], noise_std_range[1])
        for _ in range(n_points):
            noisy_y = calculate_y(np.array([[point]])) + np.random.normal(0, noise_level)
            X_noisy.append(point)
            y_noisy.append(noisy_y)
            noise_levels.append(noise_level)

    X_noisy = np.array(X_noisy).reshape(-1, 1)
    y_noisy = np.array(y_noisy).reshape(-1, 1)
    noise_levels = np.array(noise_levels)

    return X_noisy, y_noisy, noise_levels

gp_var = 1

**Automated 1D GP --> CSV Files and Plotting.**

In [3]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory in Google Drive where you want to save the results
save_dir = '/content/drive/MyDrive/Vanilla_GP_Results_1D'
os.makedirs(save_dir, exist_ok=True)


Mounted at /content/drive


In [7]:
# Function to generate a grid of points for prediction
def generate_grid(x_limits, num_points=100):
    grid = np.linspace(x_limits[0], x_limits[1], num_points)
    return grid.reshape(-1, 1)

# Function to calculate y based on the Forrester function with added homogeneous noise
def calculate_y(x, noise_std=1.0):
    term1 = (6 * x - 2) ** 2 * np.sin(12 * x - 4)
    noise = np.random.normal(0, noise_std, size=x.shape)
    return term1 + noise

# DEFINE GP MODELS + TRAINING
def train_gp_model(X, y):
    kernel = gpflow.kernels.SquaredExponential(lengthscales=0.1) # e.g., 0.3 | 0.1
    model = gpflow.models.GPR(data=(X, y), kernel=kernel, mean_function=None)
    model.likelihood.variance.assign(1.0 ** 2)  # Set variance to match the noise level
    gpflow.utilities.set_trainable(model.likelihood.variance, False)
    opt = gpflow.optimizers.Scipy()
    opt.minimize(model.training_loss, model.trainable_variables)
    return model

# Expected Improvement function for minimization
def expected_improvement(y_best, f_mean, f_var):
    variance = np.maximum(f_var, 1e-9)
    std_dev = np.sqrt(variance)
    delta = y_best - f_mean
    with np.errstate(divide='ignore'):
        Z = delta / std_dev
        ei = delta * norm.cdf(Z) + std_dev * norm.pdf(Z)
        ei[std_dev == 0.0] = 0.0
    return ei

# MAIN SCRIPT
x_limits = [0.0, 1.0]  # Limits for x

# Define the levels for x
x_manual = np.array([0.0, 0.5, 1.0])

# Generate the initial noisy data
X = x_manual.reshape(-1, 1)
y = calculate_y(X, noise_std=1.0)  # Add homogeneous noise with standard deviation of 0.1

# Train GP model
model = train_gp_model(X, y)

# Create a DataFrame to store results
results_df = pd.DataFrame(columns=['iteration', 'x1', 'y', 'ei', 'LCB', 'y_best', 'true_var', 'pred_var'])

# Run iterations
num_iterations = 20
for iteration in range(num_iterations):
    # Generate grid for prediction
    X_plot = generate_grid(x_limits, 500)

    # Predict f_mean and f_var for the model
    f_mean, f_var = model.predict_f(X_plot)
    f_mean = f_mean.numpy()
    f_var = f_var.numpy()

    # Calculate EI and LCB
    y_best = np.min(y)
    ei = expected_improvement(y_best, f_mean, f_var)
    LCB = f_mean - 1.96 * np.sqrt(f_var)
    UCB = f_mean + 1.96 * np.sqrt(f_var)

    # Calculate true variance
    y_true = calculate_y(X_plot)
    true_var = np.var(y_true - f_mean)

    # Find the point that maximizes EI
    max_ei_index = np.argmax(ei)
    x1_new_ei = X_plot[max_ei_index][0]
    y_new_ei = calculate_y(np.array([[x1_new_ei]]))[0]

    # Find the point with the lowest LCB
    min_lcb_index = np.argmin(LCB)
    x1_new_lcb = X_plot[min_lcb_index][0]
    y_new_lcb = calculate_y(np.array([[x1_new_lcb]]))[0]
    lcb_value = LCB[min_lcb_index][0]
    pred_var = f_var[min_lcb_index][0]

    # Update the training data with the new points
    X = np.vstack((X, [[x1_new_ei]]))
    y = np.vstack((y, y_new_ei))
    X = np.vstack((X, [[x1_new_lcb]]))
    y = np.vstack((y, y_new_lcb))

    # Re-train the GP model
    model = train_gp_model(X, y)

    # Create a DataFrame for the new rows and concatenate it to results_df
    new_rows = pd.DataFrame([
        {
            'iteration': iteration,
            'x1': x1_new_ei,
            'y': y_new_ei,
            'ei': ei[max_ei_index],
            'LCB': None,  # EI point does not have an LCB value
            'y_best': round(y_best, 2),
            'true_var': true_var,
            'pred_var': None,  # EI point does not have a predicted variance value
        },
        {
            'iteration': iteration,
            'x1': x1_new_lcb,
            'y': y_new_lcb,
            'ei': ei[min_lcb_index],
            'LCB': lcb_value,  # Save the LCB value
            'y_best': round(y_best, 2),
            'true_var': true_var,
            'pred_var': pred_var,  # Save the predicted variance value
        }
    ])
    results_df = pd.concat([results_df, new_rows], ignore_index=True)

    # Print the new points
    print(f"New EI point added: x1 = {x1_new_ei}, y = {y_new_ei}, ei = {ei[max_ei_index]}, y_best = {round(y_best, 2)}")
    print(f"New LCB point added: x1 = {x1_new_lcb}, y = {y_new_lcb}, ei = {ei[min_lcb_index]}, LCB = {lcb_value}, y_best = {round(y_best, 2)}")

    # Save the plot after each iteration
    fontsize = 10
    linewidth = 1.5

    # Plot preparation
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3), dpi=800)
    fig.tight_layout(pad=3.0)

    # Plot f_mean, LCB, UCB, and scatter observed points
    ax1.fill_between(X_plot.flatten(), LCB.flatten(), UCB.flatten(), color='silver', alpha=0.3)
    ax1.plot(X_plot, f_mean, color="green", linestyle='-', linewidth=linewidth, label='Mean')
    ax1.plot(X_plot, LCB, color="red", linestyle='-', linewidth=linewidth, label='LCB')
    ax1.plot(X_plot, UCB, color="blue", linestyle='-', linewidth=linewidth, label='UCB')
    ax1.set_xticks(X.flatten())
    ax1.tick_params(axis='x', labelsize=fontsize)
    ax1.tick_params(axis='y', labelsize=fontsize)
    ax1.set_xlabel("x", fontsize=fontsize)
    ax1.set_ylabel("f", fontsize=fontsize)
    ax1.grid(True, which='major', color='gray', linestyle='--')
    ax1.scatter(X, y, color="black", s=10, label='Observed Points')
    ax1.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.2f}"))
    ax1.plot(X_plot, y_true, color="black", linewidth=linewidth, label='True Function')

    # Plot EI
    ax2.plot(X_plot, ei, color="red", linewidth=linewidth, label='EI')
    ax2.set_xlabel("x", fontsize=fontsize)
    ax2.set_ylabel("EI", fontsize=fontsize)
    ax2.set_xticks(X.flatten())
    ax2.tick_params(axis='x', labelsize=fontsize)
    ax2.tick_params(axis='y', labelsize=fontsize)
    ax2.set_xlabel("x", fontsize=fontsize)
    ax2.set_ylabel("EI", fontsize=fontsize)
    ax2.grid(True, which='major', color='gray', linestyle='--')
    ax2.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.2f}"))

     # Save the plot to a file in Google Drive
    plt.savefig(os.path.join(save_dir, f'HomoNoise_Vanilla_GP_Forrester_iteration_{iteration + 1}_1Dplots.png'))
    plt.close()

    print(f"Figure saved for iteration {iteration + 1}")

# After all iterations have completed, save the final results to a CSV file in Google Drive
results_df.to_csv(os.path.join(save_dir, 'HomoNoise_Final_Vanilla_Forrester_1D_GP_Results.csv'), index=False)
print("Final results saved to Final_GP_Results.csv")

<ipython-input-7-656f7652d7be>:114: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_rows], ignore_index=True)


New EI point added: x1 = 0.2945891783567134, y = [-0.10059155], ei = [2.26470849], y_best = 1.3
New LCB point added: x1 = 0.2685370741482966, y = [-0.79269176], ei = [2.23253018], LCB = -9.446806923971435, y_best = 1.3
Figure saved for iteration 1
New EI point added: x1 = 0.30260521042084165, y = [-0.39007113], ei = [0.11585174], y_best = -0.79
New LCB point added: x1 = 0.2945891783567134, y = [1.61805619], ei = [0.11523735], LCB = -1.6937778980268101, y_best = -0.79
Figure saved for iteration 2
New EI point added: x1 = 0.3106212424849699, y = [0.69829211], ei = [0.00734714], y_best = -0.79
New LCB point added: x1 = 0.312625250501002, y = [1.2144825], ei = [0.00734511], LCB = -0.8850209927144294, y_best = -0.79
Figure saved for iteration 3
New EI point added: x1 = 0.4308617234468938, y = [-0.01576798], ei = [0.00050625], y_best = -0.79
New LCB point added: x1 = 0.3066132264529058, y = [0.74426533], ei = [0.0002541], LCB = -0.4458529746854066, y_best = -0.79
Figure saved for iteration 4